In [1]:
import pycutestmgr as cute
import time
import pdb 
import numpy as np
from numpy.linalg import norm
import matplotlib.pyplot as plt
import scipy.sparse as spc
from ipsolver import parse_cutest_like_problem, ipsolver, parse_matlab_like_problem, check_derivative
cute.clearCache('HS71')

# Import Problem

In [2]:
problems = [("CORKSCRW", {"T": 50}),  # 1
            ("COSHFUN", {"M": 20}),  # 2 --- Use barrier parameter equals to 10
            ("DIXCHLNV", {"N":100}), # 3
            ("HAGER4", {"N": 1000}), # 4
            ("HIMMELBK", {}), # 5
            ("NGONE", {"HNS": 49}), # 6 --- many iteractions
            ("OPTCNTRL", {}), # 7
            ("OPTMASS", {"N": 200}),  # 8 --- OPT 4.6e-4
            ("ORTHREGF", {"NPTS": 20}),  # 9
            ("SVANBERG", {"N": 500}),  # 10
            ("READING1", {"N": 100}), # 11  --- Fails !!! Warning --- Take a look
            ("HS7", {}), # 12
            ("HS10", {}), # 13
            ("HS11", {}), # 14
            ("HS13", {}), # 15
            ("HS14", {}), # 16
            ("HS16", {}), # 17
            ("HS17", {}), # 18
            ("HS19", {}), # 19
            ("HS20", {}), # 20
            ("HS22", {}), # 21
            ("HS24", {}), # 22
            ("HS26", {}), # 23
            ("HS28", {}), # 24
            ("HS31", {}), # 25
            ("HS32", {}), # 26
            ("HS33", {}), # 27
            ("HS39", {}), # 28
            ("HS46", {}), # 29
            ("HS51", {}), # 30
            ("HS52", {}), # 31
            ("HS53", {}), # 32
            ("HS63", {}), # 33
            ("HS64", {}), # 34
            ("HS65", {}), # 35
            ("HS70", {}), # 36
            ("HS71", {}), # 37
            ("HS72", {}), # 38
            ("HS73", {}), # 39
            ("HS74", {}), # 40
            ("HS75", {}), # 41
            ("HS77", {}), # 42
            ("HS78", {}), # 43
            ("HS79", {}), # 44
            ("HS80", {}), # 45
            ("HS81", {}), # 46
            ("HS83", {}), # 47
            ("HS84", {}), # 48
            ("HS85", {}), # 49 --- Needs to use initial barrier parameter equals to 0.001.
            ("HS86", {}), # 50
            ("HS93", {}), # 51
            ("HS95", {}), # 52
            ("HS96", {}), # 53
            ("HS97", {}), # 54 --- Needs to use initial barrier parameter 100
            ("HS98", {}), # 55 --- Needs to use initial barrier parameter 100
            ("HS99", {}), # 56 --- Fails!
            ("HS100", {}), # 57
            ("HS104", {}), # 58
            ("HS106", {}), # 60
            ("HS107", {}), # 61
            ("HS108", {}), # 62
            ("HS109", {}), # 63 
            ("HS111", {}), # 64 --- Needs to use initial barrier parameter equals to 10.
            ("HS112", {}), # 65
            ("HS113", {}), # 66
            ("HS114", {}), # 67
            ("HS116", {}), # 68
            ("HS117", {}), # 69 
            ("HS118", {}), # 70
            ("HS119", {}), # 71 
            ]

In [3]:
number = 6
name = problems[number-1][0]
cute.clearCache(name)
cute.prepareProblem(name, sifParams=problems[number-1][1])
problem = cute.importProblem(name)
info = problem.getinfo()
x0 = info["x"]
n = info["n"]
m = info["m"]
print(n, m)
v0 = info["v"]
eq_list = info["equatn"]

def convert_to_inf(bounds):
    new_bounds = np.zeros_like(bounds)
    i = 0
    for b in bounds:
        if b >= 1e20:
            new_bounds[i] = np.inf
        elif b <= -1e20:
            new_bounds[i] = -np.inf
        else:
            new_bounds[i] = b
        i += 1
    return new_bounds

# Define upper and lower bound
c_lb = convert_to_inf(info["cl"])
c_ub = convert_to_inf(info["cu"])
lb = convert_to_inf(info["bl"])
ub = convert_to_inf(info["bu"])

# Define function and constraints
def fun(x):
    return problem.obj(x)[0]

def grad(x):
    _, g1 = problem.obj(x, True)
    return g1

def lagr_hess(x, v):
    def matvec(p):
        return problem.hprod(p, x, v)
    return spc.linalg.LinearOperator((n, n), matvec=matvec)


def constr(x):
    return problem.cons(x)

def jac(x):
    _, A1 = problem.cons(x, True)
    return spc.csc_matrix(A1)

# Make initial point feasible
if number == 59:
    x0[2] = x0[2] + 6
    x0[3] = x0[3] + 6

# Parse constraints
fun, grad, new_lagr_hess, n_total_ineq, constr_ineq, \
    jac_ineq, n_total_eq, constr_eq, jac_eq, box_constraints \
        = parse_cutest_like_problem(
            fun, grad, lagr_hess, n, constr, jac,
            c_lb, c_ub, lb, ub)
        
# Remove box contraints
if number not in (59, 65):
       box_constraints.fill(0)
        
# Diferent barrier parameters
if number in (2, 64, ):
    initial_barrier_parameter = 10
elif number in (49,):
    initial_barrier_parameter = 0.001
elif number in (54, 55):
    initial_barrier_parameter = 100
else:
    initial_barrier_parameter = 0.1

(100, 1273)


# Optimize Problem

In [4]:
x, info = ipsolver(fun, grad, new_lagr_hess, n_total_ineq, constr_ineq,
                jac_ineq, n_total_eq, constr_eq, jac_eq, x0, box_constraints,


0 1 9.8765567512 2 1 100 0.855341437591 17.3286575781 -5.30224652984
1 2.0 9.75523034929 2 1 100 0.918958845506 5.52087807936 7.88337975861
2 14.0 11.730519844 2 4 100 -2.33639419717 4.18471080259 -19.0613220606
3 3.41777342803 11.730519844 2 2 100 0.847606271789 3.42342049015 17.3912297763
4 6.83554685607 13.1728826573 2 3 100 -0.335343725208 3.02691073698 29.0784801163
5 3.41777342803 13.1728826573 2 1 100 0.74108664167 2.79384155253 37.2761207201
6 6.83554685607 15.658999913 2 3 100 -1.57171034746 3.05299636455 20.1957938178
7 2.33040216778 15.658999913 2 1 100 0.567114890355 2.04404534385 15.5926143132
8 4.66080433556 13.1473227705 2 6 100 0.709213928079 2.2481004377 51.5431149222
9 8.68471076737 10.0060335892 2 10 100 -0.877699313057 1.75517235644 69.376451045
10 4.34235538368 10.0060335892 2 3 100 -2.95022281805 1.53161338975 60.4500277542
11 1.09926845656 10.0060335892 2 1 100 -0.056109817395 0.764101363461 16.3895356486
12 0.549634228279 10.0060335892 2 1 100 0.380570845075 0.5

41 0.0260505567301 0.0371391382709 2 1 100 -0.511993103712 1.62750349946e-05 -0.000122106755371
42 0.0130252783651 0.0371391382709 2 1 100 0.508207127737 1.17490841183e-05 -6.13460519914e-05
43 0.0260505567301 0.036642662184 2 1 100 -0.497144159185 1.61343866508e-05 -0.000117690648597
44 0.0130252783651 0.036642662184 2 1 100 0.511760985913 1.16320717856e-05 -6.05875246506e-05
45 0.0260505567301 0.0361583853115 2 1 100 -0.463614899269 1.59501417546e-05 -0.000107615432465
46 0.0130252783651 0.0361583853115 2 1 100 0.521748721341 1.14712255944e-05 -5.40860894327e-05
47 0.0260505567301 0.0356833289001 2 1 100 -0.443696215082 1.58256347463e-05 -0.000103381474969
48 0.0130252783651 0.0356833289001 2 1 100 0.527318287737 1.13706730798e-05 -5.33431498369e-05
49 0.0260505567301 0.0352205031539 2 1 100 -0.411519198225 1.56279570839e-05 -9.37894053952e-05
50 0.0130252783651 0.0352205031539 2 1 100 0.536652034388 1.11967131252e-05 -4.7158425207e-05
51 0.0260505567301 0.0347669837443 2 1 100 -0.38

131 0.170869257664 0.0140124921285 2 1 100 -0.543882944981 3.88790279752e-05 0.00148886754428
132 0.0854346288319 0.0140124921285 2 1 100 0.584007800105 2.24443679853e-05 0.00071675951864
133 0.170869257664 0.0135730376493 2 1 100 -1.14985939025 3.51760808784e-05 0.00152760263892
134 0.0794792704722 0.0135730376493 2 1 100 0.410950853195 1.79860414651e-05 0.000681368030736
135 0.158958540944 0.0131639232049 2 1 100 -0.708960737994 3.43097367016e-05 0.00145227297625
136 0.0794792704722 0.0131639232049 2 1 100 0.519892288349 1.97089101819e-05 0.000702004384583
137 0.158958540944 0.0127545706758 2 1 100 -1.23430844205 3.18144761329e-05 0.00148649691089
138 0.0711444026093 0.0127545706758 2 1 100 0.426397887387 1.60177890358e-05 0.000640290385888
139 0.142288805219 0.012387594984 2 1 100 -0.478977888037 3.07895068618e-05 0.00135349918608
140 0.0711444026093 0.012387594984 2 1 100 0.598839688477 1.82947814832e-05 0.000658146379862
141 0.142288805219 0.0120200741054 2 1 100 -1.04567327654 2.

214 0.0269245062308 0.00336294741145 2 1 182.112547308 0.198878750389 3.02655051571e-06 0.000381648933631
215 0.0269245062308 0.00321497282202 2 1 182.112547308 0.34971328114 3.25167441962e-06 0.00038409091954
216 0.0538490124617 0.0030670832584 2 1 202.600291365 -0.918308059815 5.48657788604e-06 0.000778107594816
217 0.0269245062308 0.0030670832584 2 1 182.112547308 0.31117377021 3.15611722417e-06 0.000386212417001
218 0.0538490124617 0.00291937992951 2 1 188.891461466 -0.76502027322 5.92239375687e-06 0.00078308272848
219 0.0269245062308 0.00291937992951 2 1 182.112547308 0.521462250352 3.61767274772e-06 0.000388894891615
220 0.0538490124617 0.00277178844328 2 1 194.824904399 -0.72239276 5.77298815546e-06 0.00078730530584
221 0.0269245062308 0.00277178844328 2 1 182.112547308 0.501675025149 3.49293507801e-06 0.000390824984001
222 0.0538490124617 0.002624440064 2 1 182.112547308 -0.162860809016 6.96978467604e-06 0.000792791825768
223 0.0269245062308 0.002624440064 2 1 182.112547308 0.7

52 0.0488730107315 0.0321578059249 2 1 100 -0.241817811112 6.738604485e-06 -0.000781818238401
53 0.0244365053657 0.0321578059249 2 1 100 0.561797674633 4.93048700051e-06 -0.00039206966394
54 0.0488730107315 0.0320571373106 2 1 100 -0.374514285609 5.92161373826e-06 -0.000778337076143
55 0.0244365053657 0.0320571373106 2 1 100 0.487383695361 4.1153042129e-06 -0.000390024762875
56 0.0488730107315 0.031957834972 2 1 100 -0.18186789133 6.78226053419e-06 -0.000776027453193
57 0.0244365053657 0.031957834972 2 1 100 0.584903439166 4.98288658525e-06 -0.000389133612704
58 0.0488730107315 0.0318600618571 2 1 100 -0.315421720157 5.94653150858e-06 -0.00077267424649
59 0.0244365053657 0.0318600618571 2 1 100 0.511237210673 4.1508212363e-06 -0.0003871752836
60 0.0488730107315 0.0317635494722 2 1 100 -0.12106363977 6.85295059531e-06 -0.000770399250866
61 0.0244365053657 0.0317635494722 2 1 100 0.609119087345 5.0663595881e-06 -0.000386280010906
62 0.0488730107315 0.0316684490289 2 1 100 -0.255190293783

...

659 0.0407245663843 0.00211865140886 2 1 100 -0.557662530764 1.23519092391e-06 7.73782807749e-05
660 0.0203622831922 0.00211865140886 2 1 100 0.629317097327 8.47186861839e-07 3.82496987428e-05
661 0.0407245663843 0.0020932511591 2 1 100 -0.592853892636 1.21352386628e-06 7.82567253159e-05
662 0.0203622831922 0.0020932511591 2 1 100 0.628830773586 8.29918896114e-07 3.88219484035e-05
663 0.0407245663843 0.00206801769655 2 1 100 -0.275039799066 1.34318683511e-06 7.85879730536e-05
664 0.0203622831922 0.00206801769655 2 1 100 0.705718894142 9.64940561359e-07 3.88470714179e-05
665 0.0407245663843 0.00204294412994 2 1 100 -0.313966750594 1.30981431364e-06 7.94918760289e-05
666 0.0203622831922 0.00204294412994 2 1 100 0.703376435005 9.36699556524e-07 3.94489031074e-05
667 0.0407245663843 0.00201803939984 2 1 100 0.057326742431 1.53553186003e-06 7.97748337734e-05
668 0.0407245663843 0.00196859512429 2 1 100 -0.00597118306201 1.5280612692e-06 8.13453407949e-05
669 0.0203622831922 0.00196859512429

738 0.00937339094714 0.00129884630011 2 1 149.410792411 0.213936587887 2.14403261611e-07 2.2131582313e-05
739 0.00937339094714 0.00129019978066 2 1 149.410792411 0.185358279497 2.13285277e-07 2.22776465484e-05
740 0.00937339094714 0.00128159689639 2 1 149.410792411 0.202637074296 2.16013439037e-07 2.22250086753e-05
741 0.00937339094714 0.00127303497259 2 1 149.410792411 0.170934740547 2.14744642033e-07 2.23745623722e-05
742 0.00937339094714 0.00126451692416 2 1 149.410792411 0.194869116562 2.17865387782e-07 2.23205717122e-05
743 0.00937339094714 0.00125604004289 2 1 149.410792411 0.159052885546 2.16350638707e-07 2.24699628868e-05
744 0.00937339094714 0.00124760727108 2 1 149.410792411 0.18969829658 2.1991126982e-07 2.24144583433e-05
745 0.00937339094714 0.00123921587123 2 1 149.410792411 0.150027064992 2.18160909295e-07 2.2564661397e-05
746 0.00937339094714 0.00123086881898 2 1 149.410792411 0.187657600055 2.22212660881e-07 2.25076743503e-05
747 0.00937339094714 0.00122256334183 2 1 14

815 0.0111938126548 0.000969096274393 2 1 230.873669227 -1.02187424325 1.77875348574e-07 2.87467140732e-05
816 0.00553635448901 0.000969096274393 2 1 167.246880825 0.312583810319 1.21911165254e-07 1.42151602954e-05
817 0.011072708978 0.000965107984439 2 1 227.896897877 -0.99039291147 1.77952785008e-07 2.83884213703e-05
818 0.00553635448901 0.000965107984439 2 1 167.246880825 0.340061821739 1.23364035885e-07 1.41552075123e-05
819 0.011072708978 0.000961134987861 2 1 230.392472486 -1.015779632 1.76625340648e-07 2.84852042548e-05
820 0.00549301555165 0.000961134987861 2 1 167.246880825 0.322620738112 1.21785685911e-07 1.41276195164e-05
821 0.0109860311033 0.000957210144946 2 1 227.436832395 -0.991301193783 1.77206499009e-07 2.82122993701e-05
822 0.00549301555165 0.000957210144946 2 1 167.246880825 0.352214468699 1.23395061674e-07 1.40651903019e-05
823 0.0109860311033 0.000953300360334 2 1 229.986533698 -1.01634947079 1.75877150772e-07 2.83145633839e-05
824 0.00544847564998 0.0009533003603

895 0.0105188363881 0.000827587529919 2 1 211.577296553 -0.866166691472 1.88402575228e-07 2.79031952712e-05
896 0.00525941819407 0.000827587529919 2 1 167.246880825 0.587687055894 1.43566098554e-07 1.3954780726e-05
897 0.0105188363881 0.000824401452268 2 1 204.601428878 -0.755973301626 1.94487326907e-07 2.78546694888e-05
898 0.00525941819407 0.000824401452268 2 1 167.246880825 0.640564254104 1.49866029267e-07 1.38962437946e-05
899 0.0105188363881 0.000821229510859 2 1 210.18629612 -0.834261720028 1.89877220553e-07 2.79367127939e-05
900 0.00525941819407 0.000821229510859 2 1 167.246880825 0.604676696631 1.45464887394e-07 1.3968433411e-05
901 0.0105188363881 0.000818073492257 2 1 209.071996309 -0.807491915135 1.90322041192e-07 2.78537063655e-05
902 0.00525941819407 0.000818073492257 2 1 167.246880825 0.617790212869 1.46118961051e-07 1.38754397114e-05
903 0.0105188363881 0.000814931340918 2 1 216.252960524 -0.889372537796 1.849114657e-07 2.79913563246e-05
904 0.00525941819407 0.0008149313

971 0.00732865309895 0.000728939134422 2 1 287.440610752 -0.950520116671 9.86620778946e-08 1.98516415496e-05
972 0.00366432654947 0.000728939134422 2 1 223.845504348 0.475844301346 7.07276572499e-08 9.93045400216e-06
973 0.00732865309895 0.000727068798107 2 1 282.867261472 -0.913923327812 1.00046140288e-07 1.98098444069e-05
974 0.00366432654947 0.000727068798107 2 1 223.845504348 0.509730503296 7.21816397991e-08 9.88454714426e-06
975 0.00732865309895 0.000725205024743 2 1 286.42025667 -0.944493105422 9.90956759116e-08 1.98681062506e-05
976 0.00366432654947 0.000725205024743 2 1 223.845504348 0.486044383754 7.13001353336e-08 9.9387765735e-06
977 0.00732865309895 0.000723349067804 2 1 281.605993475 -0.904160305042 1.00576825545e-07 1.98261258148e-05
978 0.00366432654947 0.000723349067804 2 1 223.845504348 0.520925568265 7.28520008641e-08 9.89271798834e-06
979 0.00732865309895 0.000721499660725 2 1 285.270258792 -0.936316418277 9.95770327065e-08 1.9884456123e-05
980 0.00366432654947 0.000

# Display Results

In [6]:
print('f = '+ str(info["fun"]))
print('optimality = ' + str(info["opt"]))
print('c violation = ' + str(info["constr_violation"]))
print('niter = ' + str(info["niter"]))
#print('total time = ' + str(total_time))

f = 0.28433858126
optimality = 0.000701631981912
c violation = 2.24837016993e-05
niter = 1297
